In [34]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch import tensor
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from skimage import io, color
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchmetrics import JaccardIndex, Dice
from tqdm import tqdm
from sklearn.metrics import jaccard_score
import gc

In [35]:
# Load images from folders A and B
folder_A = 'testval/A'
folder_B = 'testval/B'
folder_label = 'testval/label'

In [36]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [37]:
print(device)

cuda


In [38]:
# Function to load images from directory
def load_images_from_folder(folder, is_gray = True):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename)).astype(np.uint8)
        if img is not None:
            if is_gray:
                images.append(color.rgb2gray(img))
            else:    
                images.append(img)
    return images


In [39]:
images_A = load_images_from_folder(folder_A, is_gray=False)

In [40]:
images_B = load_images_from_folder(folder_B, is_gray=False)

In [41]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [42]:
print(len(images_A), len(images_B), len(labels)) 

1200 1200 1200


In [43]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5),
])

In [44]:
class ChangeDetectionDataset(Dataset):
    def __init__(self, images_A, images_B, labels):
        self.images_A = images_A
        self.images_B = images_B
        self.labels = labels
#         self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_A = torch.tensor(self.images_A[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        image_B = torch.tensor(self.images_B[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        label = torch.tensor(self.labels[idx], dtype=torch.float32) / 255


        return image_A, image_B, label

In [45]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    model = model.to(device)
    model.eval()
    return model

model = load_checkpoint('checkpoint.pth')
print(model)

USSFCNet(
  (Maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv1_1): First_DoubleConv(
    (conv): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (Conv1_2): First_DoubleConv(
    (conv): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplac

In [46]:
# model = USSFCNet(in_ch=3, out_ch=1, ratio=0.5)
# PATH = 'model.pth'
# model.load_state_dict(torch.load(PATH))
# model = model.to(device)
# model.eval()

In [47]:
dataset = ChangeDetectionDataset(images_A, images_B, labels)
data_loader = DataLoader(dataset, batch_size=16)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
val_total_intersections = 0
val_total_unions = 0
with torch.no_grad():
    jaccardArr = []
    for a, b, l in tqdm(data_loader):
        a, b, l = a.to(device), b.to(device), l.to(device)
        outputs = model(a,b)
        outputs = outputs.reshape((-1,l.shape[1],l.shape[2]))
        outputs_binary = (outputs>0.5) * 1
        val_total_intersections += np.logical_and(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
        val_total_unions += np.logical_or(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
        for i in range(len(outputs_binary)):
            intersection = np.logical_and(outputs_binary[i].cpu().numpy(), l[i].cpu().numpy()).sum()
            union = np.logical_or(outputs_binary[i].cpu().numpy(), l[i].cpu().numpy()).sum()
            if union != 0:
                jaccardArr.append((intersection)/(union))
            else:
                jaccardArr.append(1)
val_jaccard = val_total_intersections / val_total_unions
secondWay = np.array(jaccardArr).mean()
print(f'Validation Jaccard Index : {val_jaccard} | Jaccard Avg : {secondWay} ')

100%|██████████| 75/75 [00:26<00:00,  2.78it/s]


Validation Jaccard Index : 0.5218624045316452 | Jaccard Avg : 0.8553570429401284 
